In [1]:
import pandas as pd
import numpy as np
settings = dict(header=True,encoding="utf-8")

In [2]:
a=pd.read_csv("iso3_to_wb_name.csv").rename(columns={"country":"wb_name"})

In [3]:
b=pd.read_csv("country_codes_extra.csv")
b.columns= b.columns.str.lower()

In [4]:
r=pd.merge(a,b,on = 'iso3')
r.head()

,iso3,wb_name,short name,official name,iso2,uni,undp,faostat,gaul
0,AFG,Afghanistan,Afghanistan,The Islamic Republic of Afghanistan,AF,4,AFG,2,1
1,AGO,Angola,Angola,the Republic of Angola,AO,24,ANG,7,8
2,ALB,Albania,Albania,the Republic of Albania,AL,8,ALB,3,3
3,AND,Andorra,Andorra,the Principality of Andorra,AD,20,AND,6,7
4,ARE,United Arab Emirates,United Arab Emirates,the United Arab Emirates,AE,784,UAE,225,255


In [5]:
r.to_csv("out/iso3_WBname_iso2_uni_undp_fasotat_gaul.csv",index=False,**settings)

In [6]:
#Any spelling to WB spelling
names_to_iso=pd.read_csv("names_to_iso.csv",usecols=["country","iso3"],index_col="country",squeeze=True)
iso_to_wb = pd.read_csv("iso3_to_wb_name.csv",usecols=["country","iso3"],index_col="iso3",squeeze=True)

any_name_to_wb_name=names_to_iso.replace(iso_to_wb)
any_name_to_wb_name.index.name="any"
any_name_to_wb_name.name="wb_name"

bads =[c for c in any_name_to_wb_name if c not in iso_to_wb.unique()]
#replace bad with empty string
any_name_to_wb_name=any_name_to_wb_name.replace(bads,np.nan)

any_name_to_wb_name.to_csv("out/any_name_to_wb_name.csv",**settings)

In [7]:
#gtap to iso
gtap_to_name = pd.read_csv("gtap_to_name.csv",index_col="gtap_region",squeeze=True)
gtap_to_iso = gtap_to_name.replace(names_to_iso)
gtap_to_iso.to_csv("out/gtap_to_iso.csv",**settings)
bads = [c for c in gtap_to_iso if c not in names_to_iso.unique()] #bugs
print(bads)

['Serbia and Montenegro']


In [8]:
gtap_to_wb = gtap_to_iso.replace(iso_to_wb)
#bugs
hop =[c for c in gtap_to_wb if c not in iso_to_wb.unique()] 
hop.remove(bads[0])
gtap_to_name[hop]

gtap_region
NFK                 Norfolk Island
TKL                        Tokelau
WLF              Wallis and Futuna
SPM      Saint Pierre and Miquelon
FLK    Falkland Islands (Malvinas)
GUF                  French Guiana
ANT           Netherlands Antilles
GLP                     Guadeloupe
MSR                     Montserrat
MTQ                     Martinique
GIB                      Gibraltar
SHN                   Saint Helena
MYT                        Mayotte
REU                        Réunion
Name: name, dtype: object

In [9]:
all_gtap_to_wb_name = pd.concat([iso_to_wb,gtap_to_wb.drop(hop)]).drop_duplicates()
all_gtap_to_wb_name.index.name = "gtap_region"
all_gtap_to_wb_name.name = "wb_name"
all_gtap_to_wb_name.to_csv("out/gtap_to_wb_name.csv",**settings)